In [11]:
import os
import pandas as pd
import pydicom
import datetime
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from skimage import measure
import shutil
import cv2
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
import random
os.environ['PYTHONHASHSEED']=str(6111)
random.seed(6111)


In [12]:
print(os.listdir('../data/csv_files/diagnosis'))

['data_nonks_all_200630.csv', 'data_ks_all_200630.csv']


In [20]:
csv_path = '../data/csv_files/diagnosis/data_ks_all_200630.csv'
new_df = pd.read_csv(csv_path, delimiter=';', dtype={'sourcefile':str})
print(new_df.columns.tolist())

/Library/Python/3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['patient', 'x_case', 'x_cancer_laterality', 'x_diadate', 'x_birthdate', 'x_age_mammo', 'x_implant', 'x_examination_type', 'basename', 'full_path_clio', 'dicom_studydate', 'dicom_bitsallocated', 'dicom_bitsstored', 'dicom_bodypartexamined', 'dicom_bodypartthickness', 'dicom_breastimplantpresent', 'dicom_burnedinannotation', 'dicom_codevalue', 'dicom_compressionforce', 'dicom_detectorelementspacing', 'dicom_detectorid', 'dicom_detectortype', 'dicom_entrancedoseinmilligy', 'dicom_exposure', 'dicom_exposurecontroldescr', 'dicom_exposurecontrolmode', 'dicom_exposureinmicroampsec', 'dicom_exposurestatus', 'dicom_exposuretime', 'dicom_exposuretimeinmicrosec', 'dicom_fieldofviewdimensions', 'dicom_imagelaterality', 'dicom_imagerpixelspacing', 'dicom_institutionname', 'dicom_kvp', 'dicom_manufacturer', 'dicom_manufacturersmodelname', 'dicom_modality', 'dicom_operatorsname', 'dicom_organdose', 'dicom_organexposed', 'dicom_pixelpaddingrangelimit', 'dicom_relativexrayexposure', 'dicom_requestedpr

# exclude women with implants

In [21]:
# exclude patients with implants
print(set(new_df['dicom_viewposition'].tolist()))
print(set(new_df['dicom_breastimplantpresent'].tolist()))
print(set(new_df['dicom_codevalue'].tolist()))
print(set(new_df['x_implant'].astype(str).tolist()))

# implant patient
implant_patient_list = []
for i in range(new_df.shape[0]):
  row = new_df.loc[i,:]
  patient = str(row['patient'])
  dicom_breastimplantpresent = str(row['dicom_breastimplantpresent'])
  dicom_viewposition = str(row['dicom_viewposition'])
  dicom_codevalue = str(row['dicom_codevalue'])
  x_implant = str(row['x_implant'])
    
  if dicom_breastimplantpresent == 'YES' \
  or 'ID' in dicom_viewposition \
  or dicom_codevalue == 'R-102D5' \
  or x_implant == '1.0':
    if patient not in implant_patient_list:
      implant_patient_list.append(patient)

print('implant len:', len(implant_patient_list))


new_df = new_df[~new_df['patient'].isin(implant_patient_list)].reset_index(drop=True)
print('======= new df len:', new_df.shape[0])
print(len(set(new_df['patient'].tolist())))

print(set(new_df['dicom_viewposition'].tolist()))
print(set(new_df['dicom_breastimplantpresent'].tolist()))
print(set(new_df['dicom_codevalue'].tolist()))
print(set(new_df['x_implant'].astype(str).tolist()))

{nan, 'MLOID', 'MLO', 'CC', 'CCID'}
{nan, 'NO', 'YES'}
{'R-10242', 'R-10226'}
{'1.0', 'nan'}
implant len: 387
======= new df len: 261202
21049
{nan, 'MLO', 'CC'}
{nan, 'NO'}
{'R-10242', 'R-10226'}
{'nan'}


# exclude biopsy images

In [32]:
# exclude biopsy images
print('df len:', len(new_df))
body_parts = set(new_df['dicom_bodypartexamined'].tolist())

print(body_parts)
print('len biopsy:', len(new_df[new_df['dicom_bodypartexamined'] == 'Bröstpunktion']))

new_df2 = new_df[new_df['dicom_bodypartexamined'] != 'Bröstpunktion']
# biopsy images would have something like this- BrÃ¶stpunktion

print('\nafter excluding biopsy:')
print('len new_df2:', len(new_df2))
print(set(new_df2['dicom_bodypartexamined'].tolist()))

df len: 261202
{'MAMMOGRAFIUTREDN', 'BREAST', 'MAM KLIN', 'Bröstpunktion', 'KLINISK', 'TVABILDS_MAMMO', 'SELEKTERAD', 'MAMMOGRAFI', 'SCR 2_BILD', 'MAM KLIN H'}
len biopsy: 4

after excluding biopsy:
len new_df2: 261198
{'MAMMOGRAFIUTREDN', 'BREAST', 'MAM KLIN', 'KLINISK', 'TVABILDS_MAMMO', 'SELEKTERAD', 'MAMMOGRAFI', 'SCR 2_BILD', 'MAM KLIN H'}


# exclude mammograms that are not vendor post-processed

In [33]:
print(set(new_df['dicom_sopclassuid'].tolist()))
# new_df = new_df[new_df['dicom_sopclassuid']=='Digital Mammography X-Ray Image Storage - For Presentation'].reset_index(drop=True)


print('len new_df:', new_df.shape[0])
print('patients:', len(set(new_df['patient'].tolist())))
print(set(new_df['dicom_sopclassuid'].tolist()))

{'1.2.840.10008.5.1.4.1.1.1.2', '1.2.840.10008.5.1.4.1.1.7', '1.2.840.10008.5.1.4.1.1.1.2.1'}
len new_df: 261202
patients: 21049
{'1.2.840.10008.5.1.4.1.1.1.2', '1.2.840.10008.5.1.4.1.1.7', '1.2.840.10008.5.1.4.1.1.1.2.1'}


# exclude mammograms with aborted exposure

In [6]:
# exclude mammograms with aborted exposure
set(new_df['dicom_exposurestatus'].tolist())
# no exclusion
# images with aborted exposure would have 'ABORTED'

{'NORMAL', nan}

# exclude mammograms not taken by x-ray photoconductor

In [7]:
# exclude mammograms not taken by x-ray photoconductor
print(set(new_df['dicom_detectortype'].tolist()))
new_df = new_df[new_df['dicom_detectortype']== 'DIRECT'].reset_index(drop=True)
print(new_df.shape[0])
print(len(set(new_df['patient'].tolist())))
print(set(new_df['dicom_detectortype'].tolist()))

{'STORAGE', 'DIRECT', 'SCINTILLATOR'}
16975
2116
{'DIRECT'}


# exclude earlier mammograms when there are duplicates in the same exam

In [8]:
# exclude earlier mammograms when there are duplicates in the same exam
new_df = new_df.sort_values(by=['patient', 'dicom_studydate', 'basename'], ascending=False).reset_index(drop=True)
info_list = []
index_list = []
for i in range(new_df.shape[0]):
  row = new_df.loc[i,:]
  patient = str(row['patient'])
#   print(row['basename'])
  dicom_studydate = str(row['dicom_studydate'])
  dicom_imagelaterality = str(row['dicom_imagelaterality'])
  dicom_viewposition = str(row['dicom_viewposition'])
  lat_view = str(dicom_imagelaterality)+str(dicom_viewposition)
  info = (patient, dicom_studydate, lat_view)
#   print(info)
  if info not in info_list:
    info_list.append(info)
    index_list.append(i)
#   else:
#     print('duplicate')
new_df = new_df.loc[index_list,:].reset_index(drop=True)
print(new_df.shape[0])
print(len(set(new_df['patient'].tolist())))  

15341
2116


# exclude clinical images

In [9]:
print(set(new_df['x_examination_type']))
new_df = new_df[new_df['x_examination_type']=='Screening'].reset_index(drop=True)
print(new_df.shape[0])
print(len(set(new_df['patient'].tolist())))
print(set(new_df['x_examination_type']))
set(new_df['dicom_bodypartexamined'].tolist())

{'Screening', 'Clinical'}
10844
1327
{'Screening'}


{'BREAST'}

# exclude non ks/hologic

In [10]:
print(set(new_df['dicom_manufacturer'].tolist()))
print(set(new_df['dicom_institutionname'].tolist()))

new_df = new_df[new_df['dicom_manufacturer']=='HOLOGIC, Inc.'].reset_index(drop=True)
print(new_df.shape[0])
print(len(set(new_df['patient'].tolist())))

print(set(new_df['dicom_manufacturer'].tolist()))
print(set(new_df['dicom_institutionname'].tolist()))
ks_list = ['Karolinska Mammo lab 2', 'Karolinska Solna', 'Karolinska Mammo Lab 3', 'Karolinska Universitetssjukhuset']

new_df = new_df[new_df['dicom_institutionname'].isin(ks_list)].reset_index(drop=True)
print(new_df.shape[0])
print(len(set(new_df['patient'].tolist())))

print(set(new_df['dicom_manufacturer'].tolist()))
print(set(new_df['dicom_institutionname'].tolist()))

{'SIEMENS', 'HOLOGIC, Inc.', 'IMS s.r.l.'}
{'Danderyds Sjukhus', nan, 'Karolinska Solna', 'Your Hospital Name Here', 'Sophiahemmet', 'Danderyds Sjukhus AB', 'Karolinska Mammo Lab 3', 'SOPHIAHEMMET', 'Karolinska Universitetssjukhuset', 'Karolinska Mammo lab 2'}
10744
1306
{'HOLOGIC, Inc.'}
{'Danderyds Sjukhus', nan, 'Karolinska Solna', 'Your Hospital Name Here', 'Danderyds Sjukhus AB', 'Karolinska Mammo Lab 3', 'Karolinska Universitetssjukhuset', 'Karolinska Mammo lab 2'}
10608
1275
{'HOLOGIC, Inc.'}
{'Karolinska Universitetssjukhuset', 'Karolinska Solna', 'Karolinska Mammo lab 2', 'Karolinska Mammo Lab 3'}
